In [2]:
! pip install numpy
import numpy as np
import matplotlib.pyplot as plt

#This code is the calculation of the non-thermal and thermal electron and proton fluxes in T Tauri flares.
#The code is based on the work of Brunn et al. 2024 Appendix
#The parameters are :
# Tflare (the temperature of the flare), 
# muthinj (the ratio of the non-thermal injection energy to the peak thermal energy), 
# alpha (the power law index of the non-thermal distribution), 
# EcoE (the cut-off energy of the non-thermal distribution)

#The fiducial values of these parameters are Tflare=1E7, muthinj=3, alpha=3, EcoE=1E8.

from GeneralFunctions import BetaProton, BetaElectron
c=3E10
kEneV= 8.61*10.0**(-5)  #La constante de Boltzmann en eV*K-1

def ne10(T6):
    return 20.5*T6**-0.69
def L10(T6):
    return 7.61*T6**0.45


def nTH0(Tflare, muthinj, delta): # thermal electron density (cm-3) Brunn et al. 2024 Appendix
    n = ne10(Tflare / 1E6) * 1E10 #total electron density (cm-3) from COUP sample see Brunn et al. 2023
    return n / (1 + 3 * muthinj / (delta - 1) * np.sqrt(3 * muthinj / (2 * np.pi)) * np.exp(-3 / 2 * muthinj))  
        
def nNTH0(Tflare, muthinj, delta): # non-thermal electron density (cm-3) Brunn et al. 2024 Appendix
    nth = nTH0(Tflare, muthinj, delta)
    return nth * 3* muthinj/(delta-1) * np.sqrt(3*muthinj/(2*np.pi))*np.exp(-3/2*muthinj) 

def j0Eth(E, Tflare, muthinj, alpha, EcoE):

    return nTH0(Tflare, muthinj, alpha) * 2.0 * np.sqrt(E / np.pi) * (kEneV * Tflare) ** (-3 / 2) * np.exp(-E / (kEneV * Tflare)) * BetaElectron(E) * c / (4 * np.pi)
def j0Pth(E, Tflare, muthinj, alpha, EcoE):
    return nTH0(Tflare, muthinj, alpha) * 2.0 * np.sqrt(E / np.pi) * (kEneV * Tflare) ** (-3 / 2) * np.exp(-E / (kEneV * Tflare)) * BetaProton(E) * c / (4 * np.pi)

def jOEnth(E, Tflare, muthinj, alpha, EcoE): # non-thermal electron flux Brunn et al. 2024 Appendix
    Einj= muthinj * 3 / 2 * kEneV * Tflare # non thermal injection energy
    Nnt = np.zeros_like(E)
    mask = E >= Einj
    Nnt[mask] = nNTH0(Tflare, muthinj, alpha)*(alpha-1)/Einj #Normalization factor non thermal distribution

    return Nnt * (E / Einj) ** -alpha * BetaElectron(E) * c / (4 * np.pi) * np.exp(-E / EcoE)


def jOPnth(E, Tflare, muthinj, alpha, EcoE): # non-thermal electron flux Brunn et al. 2024 Appendix
    Einj= muthinj * 3 / 2 * kEneV * Tflare # non thermal injection energy
    Nnt = np.zeros_like(E)
    mask = E >= Einj
    Nnt[mask] = nNTH0(Tflare, muthinj, alpha)*(alpha-1)/Einj #Normalization factor non thermal distribution

    return Nnt * (E / Einj) ** -alpha * BetaProton(E) * c / (4 * np.pi) * np.exp(-E / EcoE)

'''
E=np.logspace(1, 8, 500) 
Tflare=1E7
alpha=3
muthinj=3
EcoE=1E8   
plt.plot(E, jOEnth(E, Tflare, muthinj, alpha, EcoE),color='orange', label='Non-thermal Electrons')
plt.plot(E, j0Eth(E, Tflare, muthinj, alpha, EcoE),'--',color='orange', label='Thermal Electrons')
plt.plot(E, jOPnth(E, Tflare, muthinj, alpha, EcoE),color='blue', label='Non-thermal Protons')
plt.plot(E, j0Pth(E, Tflare, muthinj, alpha, EcoE),'--',color='blue', label='Thermal Protons')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('E')
plt.ylabel('Flux')
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('$E ~ \\rm (eV)$',fontsize=18)
plt.ylabel('$j ~\\rm (particles / cm^2/str/eV/s$)',fontsize=18)
plt.ylim(1E5,1E17)
plt.show()
'''

ModuleNotFoundError: No module named 'numpy'